# **Deep Learning Exploration**

## **Load Data**

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
root_dir = "/content/drive/My Drive/"
personal_dir = 'Repo_Tesis/' 
base_dir = root_dir + personal_dir
print(base_dir)

/content/drive/My Drive/Repo_Tesis/


## **Data Augmentation**

En este apartado se genera la aumentación de datos, buscando que la aumentación sea aplicada a ambas clases se crean dos generadores con las mismas caracteristicas y posterior a eso se embeben en un iterable. Esta aumentación se le aplica a cada una de las imagenes y mascaras y se obtiene dos carpetas una con mascaras aumentadas y una con imagenes aumentadas, que posterior a esto se pueden usar para entrenar redes neuronales. 

In [2]:
import os
import shutil

# Carpeta de origen
src_folder = os.path.join(base_dir, 'Training')

# Carpeta de destino principal
dst_folder = os.path.join(base_dir, 'Train_v2')
if not os.path.exists(dst_folder):
    os.makedirs(dst_folder)

# Carpeta de destino para imágenes reales
dst_real_folder = os.path.join(dst_folder, "real_img")
if not os.path.exists(dst_real_folder):
    os.makedirs(dst_real_folder)

# Carpeta de destino para máscaras
dst_mask_folder = os.path.join(dst_folder, "mask")
if not os.path.exists(dst_mask_folder):
    os.makedirs(dst_mask_folder)

# Contadores para reenumerar
real_count = 1
mask_count = 1

# Iterar a través de las carpetas de origen
for i in range (1,5):
  tif_path = os.path.join(src_folder, "real_img", f'TIF_{i}')
  mask_path = os.path.join(src_folder, "mask", f'TIF_{i}')

    # Copiar imágenes reales a la carpeta de destino
  for img_file in os.listdir(tif_path):
    real_dst_path = os.path.join(dst_real_folder, f"{real_count}.png")
    mask_dst_path = os.path.join(dst_mask_folder, f"{mask_count}.png")
    shutil.copy2(
              os.path.join(tif_path, img_file),
              real_dst_path
              )
    shutil.copy2(
              os.path.join(mask_path, img_file),
              mask_dst_path
              )
    real_count += 1
    mask_count += 1

print("Listo!")

In [2]:
import os
import cv2
from tqdm import tqdm
from glob import glob
from albumentations import CenterCrop, RandomRotate90, GridDistortion, HorizontalFlip, VerticalFlip

output_dir = os.path.join(base_dir, 'augmented_images_v2')

def load_data():
     images = sorted([os.path.join(base_dir, 'Train_v2/real_img', i) for i in os.listdir(os.path.join(base_dir, "Train_v2/real_img"))])     
     masks = sorted([os.path.join(base_dir, 'Train_v2/mask', i) for i in os.listdir(os.path.join(base_dir, "Train_v2/mask"))])
     return images, masks

In [3]:
def augment_data(images, masks, save_path, augment=True):
    H = 256
    W = 256
    
    for x, y in tqdm(zip(images, masks), total=len(images)):
        name = x.split("/")[-1].split(".")
        """ Extracting the name and extension of the image and the mask. """
        image_name = name[0]
        image_extn = name[1]

        name = y.split("/")[-1].split(".")
        mask_name = name[0]
        mask_extn = name[1]

        """ Reading image and mask. """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        """ Augmentation """
        if augment == True:
            aug = CenterCrop(H, W, p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            aug = GridDistortion(p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']

            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x4 = augmented['image']
            y4 = augmented['mask']

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x5 = augmented['image']
            y5 = augmented['mask']

            save_images = [x, x1, x2, x3, x4, x5]
            save_masks =  [y, y1, y2, y3, y4, y5]

        else:
            save_images = [x]
            save_masks = [y]

        """ Saving the image and mask. """
        idx = 0
        for i, m in zip(save_images, save_masks):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))

            if len(images) == 1:
                tmp_img_name = f"{image_name}.{image_extn}"
                tmp_mask_name = f"{mask_name}.{mask_extn}"

            else:
                tmp_img_name = f"{image_name}_{idx}.{image_extn}"
                tmp_mask_name = f"{mask_name}_{idx}.{mask_extn}"

            image_path = os.path.join(save_path, "images", tmp_img_name)
            mask_path = os.path.join(save_path, "masks", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            idx += 1

In [7]:
images, masks = load_data()
save_path = os.path.join(base_dir, 'aum_v2')
augment_data(images, masks, save_path, augment=True)

100%|██████████| 518/518 [01:08<00:00,  7.52it/s]


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os


# Define el nombre de la carpeta donde se guardarán las imágenes generadas
output_dir = os.path.join(base_dir, 'augmented_images_v2')

# Crea las carpetas si aún no existen
if not os.path.exists(os.path.join(output_dir, 'mask')):
    os.makedirs(os.path.join(output_dir, 'mask'))
if not os.path.exists(os.path.join(output_dir, 'real_img')):
    os.makedirs(os.path.join(output_dir, 'real_img'))

real_output = os.path.join(output_dir, 'real_img')
mask_output = os.path.join(output_dir, 'mask')

data_gen_args = dict(rescale=1./255,
                     rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,
                     horizontal_flip=True,
                     vertical_flip=True,
                     fill_mode='reflect')

mask_datagen = ImageDataGenerator(**data_gen_args)
real_datagen = ImageDataGenerator(**data_gen_args)

mask_generator = mask_datagen.flow_from_directory(
    os.path.join(base_dir, 'Train_v2/mask'),
    target_size=(256, 256),
    batch_size=32,
    class_mode=None,
    seed=42)

real_generator = real_datagen.flow_from_directory(
    os.path.join(base_dir, 'Train_v2/real_img'),
    target_size=(256, 256),
    batch_size=32,
    class_mode=None,
    seed=42)



"""num_images = len(os.listdir(os.path.join(base_dir, 'Train_v2/real_img')))

for i in range(num_images):
    mask_img = mask_generator.next()[0]
    real_img = real_generator.next()[0]

    mask_img = array_to_img(mask_img)
    real_img = array_to_img(real_img)

    mask_img.save(os.path.join(mask_output, f'{i}.png'))
    real_img.save(os.path.join(real_output, f'{i}.png'))"""


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


"num_images = len(os.listdir(os.path.join(base_dir, 'Train_v2/real_img')))\n\nfor i in range(num_images):\n    mask_img = mask_generator.next()[0]\n    real_img = real_generator.next()[0]\n\n    mask_img = array_to_img(mask_img)\n    real_img = array_to_img(real_img)\n\n    mask_img.save(os.path.join(mask_output, f'{i}.png'))\n    real_img.save(os.path.join(real_output, f'{i}.png'))"

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os


# Define el nombre de la carpeta donde se guardarán las imágenes generadas
output_dir = os.path.join(base_dir, 'augmented_images')

# Crea las carpetas si aún no existen
if not os.path.exists(os.path.join(output_dir, 'mask')):
    os.makedirs(os.path.join(output_dir, 'mask'))
if not os.path.exists(os.path.join(output_dir, 'real_image')):
    os.makedirs(os.path.join(output_dir, 'real_image'))

# Define un generador de imágenes de entrenamiento para la imagen de máscara y para la imagen real
train_datagen_mask = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_datagen_real = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Carga las imágenes de máscara y de imágenes reales
train_generator_mask = train_datagen_mask.flow_from_directory(
    os.path.join(base_dir, 'Training/mask'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    seed=42)

train_generator_real = train_datagen_real.flow_from_directory(
    os.path.join(base_dir, 'Training/real_img'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    seed=42)

# Genera las imágenes aumentadas y guárdalas en las carpetas correspondientes
num_augmented_images = 10 # número de imágenes aumentadas por cada imagen original

for folder in os.listdir(os.path.join(base_dir, 'Training/mask')):
    if not os.path.exists(os.path.join(output_dir, 'mask', folder)):
        os.makedirs(os.path.join(output_dir, 'mask', folder))
    if not os.path.exists(os.path.join(output_dir, 'real_image', folder)):
        os.makedirs(os.path.join(output_dir, 'real_image', folder))
    
    for i in range(num_augmented_images):
        batch_mask, batch_real = next(zip(train_generator_mask, train_generator_real))
        augmented_images_mask = train_datagen_mask.flow(batch_mask[0], batch_mask[1]).next()[0]
        augmented_images_real = train_datagen_real.flow(batch_real[0], batch_real[1]).next()[0]

        for j in range(augmented_images_mask.shape[0]):
            Image.fromarray((augmented_images_mask[j]*255/np.max(augmented_images_mask[j])).astype('uint8')).save(os.path.join(output_dir, 'mask', folder, 'augmented_' + str(i) + str(j) + '.jpg'))
            Image.fromarray((augmented_images_real[j]*255/np.max(augmented_images_real[j])).astype('uint8')).save(os.path.join(output_dir, 'real_image', folder, 'augmented_' + str(i) + str(j) + '.jpg'))


Found 532 images belonging to 4 classes.
Found 532 images belonging to 4 classes.


In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os


# Define el nombre de la carpeta donde se guardarán las imágenes generadas
output_dir = os.path.join(base_dir, 'augmented_images')

# Crea las carpetas si aún no existen
if not os.path.exists(os.path.join(output_dir, 'mask')):
    os.makedirs(os.path.join(output_dir, 'mask'))
if not os.path.exists(os.path.join(output_dir, 'real_image')):
    os.makedirs(os.path.join(output_dir, 'real_image'))

for folder in os.listdir(os.path.join(base_dir, 'Training/mask')):
    if not os.path.exists(os.path.join(output_dir, 'mask', folder)):
        os.makedirs(os.path.join(output_dir, 'mask', folder))
    if not os.path.exists(os.path.join(output_dir, 'real_image', folder)):
        os.makedirs(os.path.join(output_dir, 'real_image', folder))

# Define un generador de imágenes de entrenamiento para la imagen de máscara y para la imagen real
train_datagen_mask = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_datagen_real = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

for i in range(1,5):
  # Carga las imágenes de máscara y de imágenes reales
  train_generator_mask = train_datagen_mask.flow_from_directory(
      os.path.join(base_dir, 'Train_v2/mask'),
      target_size=(256, 256),
      batch_size=32,
      class_mode=None,
      classes=['mask'],
      shuffle=True,
      seed=42)

  train_generator_real = train_datagen_real.flow_from_directory(
      os.path.join(base_dir, 'Train_v2/real_img'),
      target_size=(256, 256),
      batch_size=32,
      class_mode=None,
      classes=['real_img'],
      shuffle=True,
      seed=42)

  # Genera las imágenes aumentadas y guárdalas en las carpetas correspondientes
  num_augmented_images = 10 # número de imágenes aumentadas por cada imagen original
    
  for i in range(num_augmented_images):
    batch_mask, batch_real = next(zip(train_generator_mask, train_generator_real))
    augmented_images_mask = train_datagen_mask.flow(batch_mask[0], batch_mask[1]).next()[0]
    augmented_images_real = train_datagen_real.flow(batch_real[0], batch_real[1]).next()[0]

    for j in range(augmented_images_mask.shape[0]):
      Image.fromarray((augmented_images_mask[j]*255/np.max(augmented_images_mask[j])).astype('uint8')).save(os.path.join(output_dir, 'mask', folder, 'augmented_' + str(i) + str(j) + '.jpg'))
      Image.fromarray((augmented_images_real[j]*255/np.max(augmented_images_real[j])).astype('uint8')).save(os.path.join(output_dir, 'real_image', folder, 'augmented_' + str(i) + str(j) + '.jpg'))


Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


IndexError: ignored

## **Pruebas adicionales de data augmentation**

PD: No correr, esta en fase de prueba

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os

# Define el nombre de la carpeta donde se guardarán las imágenes generadas
output_dir = os.path.join(base_dir, 'augmented_images')

# Crea las carpetas si aún no existen
if not os.path.exists(os.path.join(output_dir, 'mask')):
    os.makedirs(os.path.join(output_dir, 'mask'))
if not os.path.exists(os.path.join(output_dir, 'real_image')):
    os.makedirs(os.path.join(output_dir, 'real_image'))

# Define un generador de imágenes de entrenamiento para la imagen de máscara
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


# Carga las imágenes de máscara y de imágenes reales
train_generator = train_datagen.flow_from_directory(
       os.path.join(base_dir, 'Training'),
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical', 
        classes=['mask', 'real_ima'])

Found 532 images belonging to 2 classes.


In [ ]:
# Define el nombre de la carpeta donde se guardarán las imágenes generadas
output_dir = os.path.join(base_dir, 'data_augmentation')

# Crea las carpetas si aún no existen
if not os.path.exists(os.path.join(output_dir, 'mask')):
    os.makedirs(os.path.join(output_dir, 'mask'))
if not os.path.exists(os.path.join(output_dir, 'real_image')):
    os.makedirs(os.path.join(output_dir, 'real_image'))

# Define un generador de imágenes de entrenamiento para ambas carpetas
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Carga las imágenes de máscara y de imágenes reales
train_mask_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'Training/mask'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical', 
    save_to_dir=os.path.join(output_dir, 'mask'),
    save_prefix='mask_')

train_real_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'Training/real_img'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical', 
    save_to_dir=os.path.join(output_dir, 'real_image'),
    save_prefix='real_')

# Combina los generadores de ambas carpetas en un solo generador
train_generator = zip(train_mask_generator, train_real_generator)


Found 532 images belonging to 4 classes.
Found 532 images belonging to 4 classes.


In [ ]:
# Generar y guardar imágenes con data augmentation
for i, (mask_batch, real_image_batch) in enumerate(train_generator):
    # Generar imágenes aumentadas
    mask_augmented_images = mask_datagen.flow(
        mask_batch[0], batch_size=32, save_to_dir=os.path.join(output_dir, 'mask'), save_prefix='aug_', save_format='png')
    real_image_augmented_images = real_image_datagen.flow(
        real_image_batch[0], batch_size=32, save_to_dir=os.path.join(output_dir, 'real_image'), save_prefix='aug_', save_format='png')

    # Guardar imágenes aumentadas
    for j in range(len(mask_batch[0])):
        mask_augmented_image = mask_augmented_images[j]
        real_image_augmented_image = real_image_augmented_images[j]
        mask_orig_image_path = mask_orig_images.filenames[i * 32 + j]
        real_image_orig_image_path = real_image_orig_images.filenames[i * 32 + j]
        mask_orig_image_name = os.path.basename(mask_orig_image_path)
        real_image_orig_image_name = os.path.basename(real_image_orig_image_path)
        mask_orig_image_dir = os.path.dirname(mask_orig_image_path)
        real_image_orig_image_dir = os.path.dirname(real_image_orig_image_path)

        mask_augmented_image_path = os.path.join(
            output_dir, 'mask', mask_orig_image_dir, 'aug_' + mask_orig_image_name)
        real_image_augmented_image_path = os.path.join(
            output_dir, 'real_image', real_image_orig_image_dir, 'aug_' + real_image_orig_image_name)

        mask_augmented_image.save(mask_augmented_image_path)
        real_image_augmented_image.save(real_image_augmented_image_path)

    # Salir del ciclo después de generar y guardar 100 imágenes aumentadas
    if i >= 3:
        break
